### Loading data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, recall_score,roc_auc_score, confusion_matrix, f1_score
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC

import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Exp_Proj_5490/Project/clean_df_copy.csv')

In [ ]:
df1

,job title,job function,job category,job group,pay level,work city,work country,work region,gender,tenure bucket,generation,work structure,vol_churn,compa bucket
0,Managing Principal Engineer,Engineering,Management,Management,M4,Fremont,United States,Americas,M,5,Boomer,Onsite,1,75-100%
1,Staff Engineer,Engineering,Engineering Professional,Professional,P4,Remote US,United States,Americas,M,5,Boomer,Remote,0,100%+
2,Sr Staff Program/Project Manager,Information Technology,Professional,Professional,P5,Fremont,United States,Americas,F,5,Gen X,Onsite,0,75-100%
3,Technologist,Engineering,Engineering Professional,Professional,P7,Longmont,United States,Americas,M,5,Boomer,Onsite,0,75-100%
4,Sr Engineering Specialist,Engineering Services,Operations Support,Support,S5,Fremont,United States,Americas,M,5,Gen X,Onsite,0,100%+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17523,Sr Manager,Information Technology,Management,Management,M4,Longmont,United States,Americas,M,5,Gen X,Onsite,1,75-100%
17524,Engineering Specialist IV,Engineering Services,Operations Support,Support,S4,Fremont,United States,Americas,M,5,Boomer,Onsite,0,75-100%
17525,Sr Engineer,Engineering,Engineering Professional,Professional,P3,Fremont,United States,Americas,F,4,Boomer,Onsite,1,75-100%
17526,Sr Staff Engineer,Engineering,Engineering Professional,Professional,P5,Longmont,United States,Americas,M,5,Boomer,Onsite,0,75-100%


### Recursive Feature training

In [ ]:
df = df1.copy()

In [ ]:
df_log = df

In [ ]:
# dummying
types = df_log.dtypes

cats = []
num = []

for i in range(len(types)):
    if types[i] == object:
        cats.append(types.index[i])
    else:
        num.append(types.index[i])


# dummying
cat_vars = cats
for var in cat_vars:
    cat_list = pd.get_dummies(df_log[var], dtype=float)
    cat_list.columns = [var + '_' + str(col) for col in cat_list.columns]
    df_log = pd.concat([df_log, cat_list], axis=1)
    df_log.drop(columns=[var], inplace=True)

df_log.reset_index(drop=True, inplace=True)

In [ ]:
# RFE
feature_df = df_log[['tenure bucket',
 'job title_Account Director',
 'job title_Account Manager I',
 'job title_Account Manager II',
 'job title_Accountant I',
 'job title_Accountant II',
 'job title_Administrative Assistant',
 'job title_Administrator I',
 'job title_Administrator II',
 'job title_Analyst I',
 'job title_Artist II',
 'job title_Assistant General Counsel',
 'job title_Assistant IV',
 'job title_Counsel II',
 'job title_Engineer I',
 'job title_Engineering Specialist II',
 'job title_Engineering Specialist III',
 'job title_Engineering Specialist IV',
 'job title_Managing Principal Engineer',
 'job title_Principal Analyst',
 'job title_Principal Product Manager',
 'job title_Representative III',
 'job title_Sr Account Manager',
 'job title_Sr Artist',
 'job title_Sr Engineer',
 'job title_Sr Engineering Specialist',
 'job title_Sr Program/Project Manager',
 'job title_Sr Team Lead',
 'job title_Staff Analyst',
 'job title_Staff Business Partner',
 'job title_Staff Commodity Manager',
 'job title_Supervisor II',
 'job function_Engineering',
 'job function_Engineering Services',
 'job function_Finance/Accounting',
 'job function_Human Resources',
 'job function_Information Technology',
 'job function_Legal',
 'job function_Manufacturing',
 'job function_Marketing',
 'job function_Materials',
 'job function_Quality',
 'job function_Sales',
 'job category_Management',
 'job category_Sales Management',
 'job category_Support',
 'job group_Management',
 'job group_Professional',
 'job group_Support',
 'pay level_M2',
 'pay level_M4',
 'pay level_M5',
 'pay level_P1',
 'pay level_P2',
 'pay level_P6',
 'pay level_P8',
 'pay level_S1',
 'pay level_S2',
 'pay level_S3',
 'pay level_S4',
 'pay level_SM5',
 'work city_Guadalajara',
 'work city_Johor',
 'work city_Korat',
 'work city_Longmont',
 'work city_Munich',
 'work city_Normandale',
 'work city_Ontario',
 'work city_Penang',
 'work city_Portland',
 'work city_Remote US',
 'work city_Shakopee',
 'work city_Woodlands  W1',
 'work city_Woodlands  W2',
 'work country_China',
 'work country_India',
 'work country_Malaysia',
 'work country_Thailand',
 'work country_United Kingdom',
 'work country_United States',
 'work region_Americas',
 'work region_Asia Pacific',
 'work region_EMEA',
 'gender_D',
 'generation_Boomer',
 'generation_Gen X',
 'generation_Gen Z',
 'generation_Millennial',
 'work structure_Hybrid',
 'work structure_Onsite',
 'work structure_Remote',
 'compa bucket_100%+',
 'compa bucket_50-75%',
 'compa bucket_75-100%']].copy()

In [ ]:
feature_df.shape

(17528, 94)

#### Splitting Data into Training and Test sets

In [ ]:
X= feature_df
y= df_log['vol_churn']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y,
                                                    shuffle=True,
                                                    random_state = 42)


In [ ]:
# Instantiate the Random Forest Classifier model
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model using the training set
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
pred_test_rf = rf_classifier.predict(X_test)
pred_test_proba_rf = rf_classifier.predict_proba(X_test)[:, 1]

# Calculate metrics on the test set
test_accuracy_rf = accuracy_score(y_test, pred_test_rf)
test_recall_rf = recall_score(y_test, pred_test_rf, average='weighted')

### RF testing w New Data ##

In [ ]:
active_df = pd.read_csv('/content/drive/MyDrive/Exp_Proj_5490/Project/clean_df_copy.csv')

In [ ]:
active = active_df.copy()

In [ ]:
# copying and remove anon_id to add bad after processing
anon_id_column = active['anon id']
active = active.drop(columns=['anon id'])

In [ ]:
# Prepping/dummying new data
types = active.dtypes

cats = []
num = []

for i in range(len(types)):
    if types[i] == object:
        cats.append(types.index[i])
    else:
        num.append(types.index[i])


cat_vars = cats
for var in cat_vars:
    cat_list = pd.get_dummies(active[var], dtype=float)
    cat_list.columns = [var + '_' + str(col) for col in cat_list.columns]
    active = pd.concat([active, cat_list], axis=1)
    active.drop(columns=[var], inplace=True)

active.reset_index(drop=True, inplace=True)

In [ ]:
# removed these columns as well bc there were no active employees under any of these
remove = ['job title_Engineering Technician I', 'job title_Engineering Technician II',
'job title_Sr Representative', 'job title_Staff Representative', 'work city_Bangalore', 'work city_Cupertino']

In [ ]:
feature_df_active = active[['tenure bucket',
 'job title_Account Director',
 'job title_Account Manager I',
 'job title_Account Manager II',
 'job title_Accountant I',
 'job title_Accountant II',
 'job title_Administrative Assistant',
 'job title_Administrator I',
 'job title_Administrator II',
 'job title_Analyst I',
 'job title_Artist II',
 'job title_Assistant General Counsel',
 'job title_Assistant IV',
 'job title_Counsel II',
 'job title_Engineer I',
 'job title_Engineering Specialist II',
 'job title_Engineering Specialist III',
 'job title_Engineering Specialist IV',
 'job title_Managing Principal Engineer',
 'job title_Principal Analyst',
 'job title_Principal Product Manager',
 'job title_Representative III',
 'job title_Sr Account Manager',
 'job title_Sr Artist',
 'job title_Sr Engineer',
 'job title_Sr Engineering Specialist',
 'job title_Sr Program/Project Manager',
 'job title_Sr Team Lead',
 'job title_Staff Analyst',
 'job title_Staff Business Partner',
 'job title_Staff Commodity Manager',
 'job title_Supervisor II',
 'job function_Engineering',
 'job function_Engineering Services',
 'job function_Finance/Accounting',
 'job function_Human Resources',
 'job function_Information Technology',
 'job function_Legal',
 'job function_Manufacturing',
 'job function_Marketing',
 'job function_Materials',
 'job function_Quality',
 'job function_Sales',
 'job category_Management',
 'job category_Sales Management',
 'job category_Support',
 'job group_Management',
 'job group_Professional',
 'job group_Support',
 'pay level_M2',
 'pay level_M4',
 'pay level_M5',
 'pay level_P1',
 'pay level_P2',
 'pay level_P6',
 'pay level_P8',
 'pay level_S1',
 'pay level_S2',
 'pay level_S3',
 'pay level_S4',
 'pay level_SM5',
 'work city_Guadalajara',
 'work city_Johor',
 'work city_Korat',
 'work city_Longmont',
 'work city_Munich',
 'work city_Normandale',
 'work city_Ontario',
 'work city_Penang',
 'work city_Portland',
 'work city_Remote US',
 'work city_Shakopee',
 'work city_Woodlands  W1',
 'work city_Woodlands  W2',
 'work country_China',
 'work country_India',
 'work country_Malaysia',
 'work country_Thailand',
 'work country_United Kingdom',
 'work country_United States',
 'work region_Americas',
 'work region_Asia Pacific',
 'work region_EMEA',
 'gender_D',
 'generation_Boomer',
 'generation_Gen X',
 'generation_Gen Z',
 'generation_Millennial',
 'work structure_Hybrid',
 'work structure_Onsite',
 'work structure_Remote',
 'compa bucket_100%+',
 'compa bucket_50-75%',
 'compa bucket_75-100%']].copy()

In [ ]:
# Make predictions on the future data
pred_future_rf = rf_classifier.predict(feature_df_active)
pred_proba_future_rf = rf_classifier.predict_proba(feature_df_active)[:, 1]


In [ ]:
# adding churn prob column
feature_df_active['churn_probability'] = pred_proba_future_rf

In [ ]:
# adding those who are likely to churn w a threshold of 70%
threshold = 0.7
feature_df_active['likely_to_churn'] = pred_proba_future_rf > threshold

In [ ]:
# adding anon id back so they can figure out who is likely
feature_df_active['anon id'] = anon_id_column

In [ ]:
# new DF with only relevant columns
selected_columns = ['anon id', 'churn_probability', 'likely_to_churn']
final_predictions = feature_df_active[selected_columns]

In [ ]:
final_predictions['likely_to_churn'].value_counts()

likely_to_churn
False    11405
True       263
Name: count, dtype: int64

### XGB Training

In [ ]:
df = df1.copy()

In [ ]:
df_log = df

In [ ]:
# dummying
types = df_log.dtypes

cats = []
num = []

for i in range(len(types)):
    if types[i] == object:
        cats.append(types.index[i])
    else:
        num.append(types.index[i])


# dummying
cat_vars = cats
for var in cat_vars:
    cat_list = pd.get_dummies(df_log[var], dtype=float)
    cat_list.columns = [var + '_' + str(col) for col in cat_list.columns]
    df_log = pd.concat([df_log, cat_list], axis=1)
    df_log.drop(columns=[var], inplace=True)

df_log.reset_index(drop=True, inplace=True)

In [ ]:
# RFE
feature_df = df_log[['tenure bucket',
 'job title_Account Director',
 'job title_Account Manager I',
 'job title_Account Manager II',
 'job title_Accountant I',
 'job title_Accountant II',
 'job title_Administrative Assistant',
 'job title_Administrator I',
 'job title_Administrator II',
 'job title_Analyst I',
 'job title_Artist II',
 'job title_Assistant General Counsel',
 'job title_Assistant IV',
 'job title_Counsel II',
 'job title_Engineer I',
 'job title_Engineering Specialist II',
 'job title_Engineering Specialist III',
 'job title_Engineering Specialist IV',
 'job title_Managing Principal Engineer',
 'job title_Principal Analyst',
 'job title_Principal Product Manager',
 'job title_Representative III',
 'job title_Sr Account Manager',
 'job title_Sr Artist',
 'job title_Sr Engineer',
 'job title_Sr Engineering Specialist',
 'job title_Sr Program/Project Manager',
 'job title_Sr Team Lead',
 'job title_Staff Analyst',
 'job title_Staff Business Partner',
 'job title_Staff Commodity Manager',
 'job title_Supervisor II',
 'job function_Engineering',
 'job function_Engineering Services',
 'job function_Finance/Accounting',
 'job function_Human Resources',
 'job function_Information Technology',
 'job function_Legal',
 'job function_Manufacturing',
 'job function_Marketing',
 'job function_Materials',
 'job function_Quality',
 'job function_Sales',
 'job category_Management',
 'job category_Sales Management',
 'job category_Support',
 'job group_Management',
 'job group_Professional',
 'job group_Support',
 'pay level_M2',
 'pay level_M4',
 'pay level_M5',
 'pay level_P1',
 'pay level_P2',
 'pay level_P6',
 'pay level_P8',
 'pay level_S1',
 'pay level_S2',
 'pay level_S3',
 'pay level_S4',
 'pay level_SM5',
 'work city_Guadalajara',
 'work city_Johor',
 'work city_Korat',
 'work city_Longmont',
 'work city_Munich',
 'work city_Normandale',
 'work city_Ontario',
 'work city_Penang',
 'work city_Portland',
 'work city_Remote US',
 'work city_Shakopee',
 'work city_Woodlands  W1',
 'work city_Woodlands  W2',
 'work country_China',
 'work country_India',
 'work country_Malaysia',
 'work country_Thailand',
 'work country_United Kingdom',
 'work country_United States',
 'work region_Americas',
 'work region_Asia Pacific',
 'work region_EMEA',
 'gender_D',
 'generation_Boomer',
 'generation_Gen X',
 'generation_Gen Z',
 'generation_Millennial',
 'work structure_Hybrid',
 'work structure_Onsite',
 'work structure_Remote',
 'compa bucket_100%+',
 'compa bucket_50-75%',
 'compa bucket_75-100%']].copy()

In [ ]:
feature_df.shape

(17528, 94)

In [ ]:
X= feature_df
y= df_log['vol_churn']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y,
                                                    shuffle=True,
                                                    random_state = 42)


In [ ]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Fit the model using the training set
xgb.fit(X_train, y_train)

# Make predictions on the test set
pred_test = xgb.predict(X_test)
pred_test_proba = xgb.predict_proba(X_test)[:, 1]

# Calculate metrics on the test set
accuracy_test = accuracy_score(y_test, pred_test)
roc_auc_test = roc_auc_score(y_test, pred_test_proba)

### XGB testing with New Data ##

In [ ]:
active_df = pd.read_csv('/content/drive/MyDrive/Exp_Proj_5490/Project/active_clean_df_copy.csv')

In [ ]:
active = active_df.copy()

In [ ]:
# copying and remove anon_id to add bad after processing
anon_id_column = active['anon id']
active = active.drop(columns=['anon id'])

In [ ]:
# Prepping/dummying new data
types = active.dtypes

cats = []
num = []

for i in range(len(types)):
    if types[i] == object:
        cats.append(types.index[i])
    else:
        num.append(types.index[i])


cat_vars = cats
for var in cat_vars:
    cat_list = pd.get_dummies(active[var], dtype=float)
    cat_list.columns = [var + '_' + str(col) for col in cat_list.columns]
    active = pd.concat([active, cat_list], axis=1)
    active.drop(columns=[var], inplace=True)

active.reset_index(drop=True, inplace=True)

In [ ]:
# removed these columns as well bc there were no active employees under any of these
remove = ['job title_Engineering Technician I', 'job title_Engineering Technician II',
'job title_Sr Representative', 'job title_Staff Representative', 'work city_Bangalore', 'work city_Cupertino']

In [ ]:
feature_df_active = active[['tenure bucket',
 'job title_Account Director',
 'job title_Account Manager I',
 'job title_Account Manager II',
 'job title_Accountant I',
 'job title_Accountant II',
 'job title_Administrative Assistant',
 'job title_Administrator I',
 'job title_Administrator II',
 'job title_Analyst I',
 'job title_Artist II',
 'job title_Assistant General Counsel',
 'job title_Assistant IV',
 'job title_Counsel II',
 'job title_Engineer I',
 'job title_Engineering Specialist II',
 'job title_Engineering Specialist III',
 'job title_Engineering Specialist IV',
 'job title_Managing Principal Engineer',
 'job title_Principal Analyst',
 'job title_Principal Product Manager',
 'job title_Representative III',
 'job title_Sr Account Manager',
 'job title_Sr Artist',
 'job title_Sr Engineer',
 'job title_Sr Engineering Specialist',
 'job title_Sr Program/Project Manager',
 'job title_Sr Team Lead',
 'job title_Staff Analyst',
 'job title_Staff Business Partner',
 'job title_Staff Commodity Manager',
 'job title_Supervisor II',
 'job function_Engineering',
 'job function_Engineering Services',
 'job function_Finance/Accounting',
 'job function_Human Resources',
 'job function_Information Technology',
 'job function_Legal',
 'job function_Manufacturing',
 'job function_Marketing',
 'job function_Materials',
 'job function_Quality',
 'job function_Sales',
 'job category_Management',
 'job category_Sales Management',
 'job category_Support',
 'job group_Management',
 'job group_Professional',
 'job group_Support',
 'pay level_M2',
 'pay level_M4',
 'pay level_M5',
 'pay level_P1',
 'pay level_P2',
 'pay level_P6',
 'pay level_P8',
 'pay level_S1',
 'pay level_S2',
 'pay level_S3',
 'pay level_S4',
 'pay level_SM5',
 'work city_Guadalajara',
 'work city_Johor',
 'work city_Korat',
 'work city_Longmont',
 'work city_Munich',
 'work city_Normandale',
 'work city_Ontario',
 'work city_Penang',
 'work city_Portland',
 'work city_Remote US',
 'work city_Shakopee',
 'work city_Woodlands  W1',
 'work city_Woodlands  W2',
 'work country_China',
 'work country_India',
 'work country_Malaysia',
 'work country_Thailand',
 'work country_United Kingdom',
 'work country_United States',
 'work region_Americas',
 'work region_Asia Pacific',
 'work region_EMEA',
 'gender_D',
 'generation_Boomer',
 'generation_Gen X',
 'generation_Gen Z',
 'generation_Millennial',
 'work structure_Hybrid',
 'work structure_Onsite',
 'work structure_Remote',
 'compa bucket_100%+',
 'compa bucket_50-75%',
 'compa bucket_75-100%']].copy()

In [ ]:
# Make predictions on the future data
pred_future_xgb = xgb.predict(feature_df_active)
pred_proba_future_xgb = xgb.predict_proba(feature_df_active)[:, 1]


In [ ]:
# adding churn prob column
feature_df_active['churn_probability'] = pred_proba_future_xgb

In [ ]:
# adding those who are likely to churn w a threshold of 70%
threshold = 0.6
feature_df_active['likely_to_churn'] = pred_proba_future_xgb > threshold

In [ ]:
# adding anon id back so they can figure out who is likely
feature_df_active['anon id'] = anon_id_column

In [ ]:
# new DF with only relevant columns
selected_columns = ['anon id', 'churn_probability', 'likely_to_churn']
final_predictions = feature_df_active[selected_columns]

#### Final Predictions

In [ ]:
final_predictions['likely_to_churn'].value_counts()

likely_to_churn
False    11150
True       518
Name: count, dtype: int64

In [ ]:
final_predictions.head(20)

,anon id,churn_probability,likely_to_churn
0,1113,0.023212,False
1,1115,0.134323,False
2,1116,0.218868,False
3,1120,0.014685,False
4,1126,0.009545,False
5,1131,0.012131,False
6,1132,0.012131,False
7,1133,0.109297,False
8,1149,0.218868,False
9,1151,0.157529,False
